In [49]:
import pandera as pa
import pandas as pd
from pandera import Column, DataFrameSchema, Check, Index
from pandera.errors import SchemaErrors

file_path = r"C:\ZC\Data Governance\DataGovernanceWorkflow\data\ssh_logs_processed.csv"
df = pd.read_csv(file_path)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84379 entries, 0 to 84378
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      84379 non-null  object
 1   Time      84379 non-null  object
 2   IP        84379 non-null  object
 3   Port      84379 non-null  int64 
 4   Username  84374 non-null  object
 5   Password  84232 non-null  object
 6   Country   84379 non-null  object
 7   City      84379 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.2+ MB


In [51]:
df.describe(include='all')

,Date,Time,IP,Port,Username,Password,Country,City
count,84379,84379,84379,84379.000000,84374,84232,84379,84379
unique,9,50909,1423,NaN,1138,21944,100,609
top,8/3/2024,17:17:37,192.3.251.75,NaN,root,123456,Australia,Sydney
freq,19184,44,14575,NaN,48886,7055,21322,18816
mean,NaN,NaN,NaN,46616.963605,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,9241.375268,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,1056.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,39825.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,47096.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,54160.000000,NaN,NaN,NaN,NaN


Clean the Data

In [52]:
df.drop_duplicates(inplace=True)

Check for missing values

In [53]:
if df.isnull().sum().sum() > 0:
    dtype_map = df.dtypes.to_dict()
    num_cols = [col for col, dt in dtype_map.items() if pd.api.types.is_numeric_dtype(dt)]
    cat_cols = [col for col, dt in dtype_map.items() if pd.api.types.is_object_dtype(dt)]

    #fill num cols null with median 
    for col in num_cols:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

    #fill categorical nulls with mode and normalized strings
    for col in cat_cols:
        if df[col].mode().empty:
            df[col] = df[col].fillna('unknown')
        else:
            mode_val = df[col].mode()[0]
            df[col] = df[col].fillna(mode_val)
        df[col] = df[col].str.strip().str.lower()


Handle outliers in numerical cols

In [54]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Remove outliers
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [55]:
print("Sample of cleaned dataset:")
print(df.head())

Sample of cleaned dataset:
        Date      Time               IP   Port   Username   Password  \
0  7/31/2024  20:22:06    85.209.11.227  29628      admin      admin   
2  7/31/2024  20:34:43   146.70.121.173  31346       sshd          1   
3  7/31/2024  20:48:15  181.176.161.157  51706        msf        msf   
4  7/31/2024  20:48:21  181.176.161.157  63288        bom        bom   
5  7/31/2024  20:48:27  181.176.161.157  62864  testuser1  testuser1   

          Country       City  
0          russia     moscow  
2  united kingdom  stretford  
3            peru     huaraz  
4            peru     huaraz  
5            peru     huaraz  


In [ ]:
schema = pa.DataFrameSchema(
    {
        "Date": Column(
            pa.DateTime,
            coerce=True,
            nullable=True,
        ),
        "Time": Column(
            pa.String,  
            nullable=True,
        ),
        "IP": Column(
            str,
            nullable=True,
            checks=Check.str_matches(
                r"^((25[0-5]|2[0-4]\d|[01]?\d\d?)\.){3}"
                r"(25[0-5]|2[0-4]\d|[01]?\d\d?)$"
            ),
        ),
        "Port": Column(
            int,
            nullable=True,
            checks=Check.in_range(1, 65535),
        ),
        "Username": Column(
            str,
            nullable=True,
            checks=Check.str_length(min_value=1),
        ),
        "Password": Column(
            str,
            nullable=True,
            checks=Check.str_length(min_value=1), 
        ),
        "Country": Column(
            str,
            nullable=True,
            checks=Check.str_length(min_value=1),
        ),
        "City": Column(
            str,
            nullable=True,
            checks=Check.str_length(min_value=1),
        ),
    }
)

try:
    validated_df = schema.validate(df, lazy=True)
    print("✅ Validation passed.")
    print(validated_df)
except SchemaErrors as err:
    print("❌ Validation failed. Issues found:")
    print(err.failure_cases)


✅ Validation passed.
            Date      Time               IP   Port   Username     Password  \
0     2024-07-31  20:22:06    85.209.11.227  29628      admin        admin   
2     2024-07-31  20:34:43   146.70.121.173  31346       sshd            1   
3     2024-07-31  20:48:15  181.176.161.157  51706        msf          msf   
4     2024-07-31  20:48:21  181.176.161.157  63288        bom          bom   
5     2024-07-31  20:48:27  181.176.161.157  62864  testuser1    testuser1   
...          ...       ...              ...    ...        ...          ...   
84374 2024-08-16  10:09:27  103.178.153.217  46594       root  asdfg!@#123   
84375 2024-08-16  10:09:30     192.3.251.75  57320       root     11041987   
84376 2024-08-16  10:09:34     192.3.251.75  57326       root       110389   
84377 2024-08-16  10:09:35  103.178.153.217  43608       root       passwd   
84378 2024-08-16  10:09:37     192.3.251.75  57334       root       110287   

              Country       City  
0      